In [1]:
# --- 匯入套件 ---
import os
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

In [2]:
# -------------------------------
# Step 1: 讀取多個 CSV
# -------------------------------
data_dir = "./dance_csv/"  # 改成你 CSV 資料夾
all_poses = []

def parse_point(s):
    s = s.strip("()")
    parts = [float(p.strip(" '")) for p in s.split(",")]
    return parts

for file in os.listdir(data_dir):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(data_dir, file))
        pose_cols = [c for c in df.columns if c != "frame"]
        poses = []
        for _, row in df.iterrows():
            pose = []
            for c in pose_cols:
                pose += parse_point(row[c])
            poses.append(pose)
        all_poses.append(np.array(poses))

all_poses = np.concatenate(all_poses, axis=0)
print("骨架資料 shape:", all_poses.shape)

骨架資料 shape: (271194, 99)


In [3]:
# -------------------------------
# Step 2: 標準化
# -------------------------------
scaler = StandardScaler()
all_poses = scaler.fit_transform(all_poses)

In [4]:
# -------------------------------
# Step 3: Dataset / DataLoader
# -------------------------------
class PoseDataset(Dataset):
    def __init__(self, data):
        self.data = torch.tensor(data, dtype=torch.float32)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]

dataset = PoseDataset(all_poses)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [5]:
# -------------------------------
# Step 4: 定義 VQ-VAE
# -------------------------------
class VectorQuantizer(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.num_embeddings = num_embeddings
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.embedding.weight.data.uniform_(-0.1, 0.1)

    def forward(self, x):
        distances = (
            torch.sum(x**2, dim=1, keepdim=True)
            + torch.sum(self.embedding.weight**2, dim=1)
            - 2 * torch.matmul(x, self.embedding.weight.t())
        )
        encoding_indices = torch.argmin(distances, dim=1)
        quantized = self.embedding(encoding_indices)
        return quantized, encoding_indices

class VQVAE(nn.Module):
    def __init__(self, input_dim=99, hidden_dim=256, latent_dim=64, num_embeddings=128):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim)
        )
        self.vq = VectorQuantizer(num_embeddings, latent_dim)
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        z_q, indices = self.vq(z)
        x_recon = self.decoder(z_q)
        return x_recon, indices, z, z_q

In [6]:
# -------------------------------
# Step 5: 訓練
# -------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
model = VQVAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()

num_epochs = 30

for epoch in range(num_epochs):
    total_loss = 0
    epoch_indices = []

    for batch in dataloader:
        batch = batch.to(device)
        optimizer.zero_grad()
        x_recon, indices, z, z_q = model(batch)
        recon_loss = loss_fn(x_recon, batch)
        vq_loss = torch.mean((z_q.detach() - z)**2) + 1.0 * torch.mean((z_q - z.detach())**2)
        loss = recon_loss + vq_loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # 收集這個 batch 的 codebook index
        epoch_indices.extend(indices.cpu().numpy().tolist())

    # 每個 epoch 統計 codebook 使用率
    unique, counts = np.unique(epoch_indices, return_counts=True)
    dist = dict(zip(unique, counts))
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}")
    print("Codebook 使用率:", dist)

Epoch 1/30, Loss: 0.8500
Codebook 使用率: {0: 49, 1: 210, 2: 175, 3: 25, 4: 15, 5: 21, 6: 243, 7: 231, 8: 155, 9: 22, 10: 60, 11: 47, 12: 145, 13: 29, 14: 124, 15: 279, 16: 80, 17: 205, 18: 18, 19: 365, 20: 119, 21: 521, 22: 189392, 23: 80, 24: 63, 25: 64, 26: 527, 27: 316, 28: 20, 29: 146, 30: 59, 31: 34, 32: 31, 33: 533, 34: 42, 35: 133, 36: 873, 37: 42, 38: 100, 39: 75, 40: 155, 41: 37, 42: 3513, 43: 342, 44: 48, 45: 544, 46: 331, 47: 201, 48: 52, 49: 245, 50: 38, 51: 319, 52: 15, 53: 377, 54: 61, 55: 308, 56: 291, 57: 86, 58: 16, 59: 305, 60: 65, 61: 26, 62: 40, 63: 57, 64: 287, 65: 621, 66: 857, 67: 181, 68: 592, 69: 126, 70: 85, 71: 187, 72: 311, 73: 58, 74: 37, 75: 17, 76: 22, 77: 34, 78: 34, 79: 24, 80: 364, 81: 540, 82: 1247, 83: 191, 84: 505, 85: 266, 86: 82, 87: 797, 88: 8, 89: 145, 90: 258, 91: 523, 92: 143, 93: 55, 94: 290, 95: 19, 96: 198, 97: 87, 98: 55, 99: 49, 100: 184, 101: 6, 102: 54861, 103: 496, 104: 222, 105: 60, 106: 388, 107: 19, 108: 21, 109: 141, 110: 40, 111: 46

In [7]:
# -------------------------------
# Step 6: 用模型生成符號序列
# -------------------------------
model.eval()
all_indices = []

with torch.no_grad():
    for batch in dataloader:
        batch = batch.to(device)
        _, indices, _, _ = model(batch)
        all_indices.extend(indices.cpu().numpy().tolist())

In [8]:
# -------------------------------
# Step 7: 轉成符號序列 (A-Z, a-z, 0-9)
# -------------------------------
def index_to_symbol(i):
    symbols = [chr(c) for c in range(65, 91)] + \
              [chr(c) for c in range(97, 123)] + \
              [str(d) for d in range(10)]
    return symbols[i % len(symbols)]

symbol_sequence = [index_to_symbol(i) for i in all_indices]

In [9]:
# -------------------------------
# Step 8: 儲存 JSON
# -------------------------------
output_path = "symbol_sequences06.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(symbol_sequence, f, indent=2, ensure_ascii=False)

print(f"✅ 訓練完成並儲存符號序列：{output_path}")
print("符號序列預覽：", "".join(symbol_sequence[:200]))

✅ 訓練完成並儲存符號序列：symbol_sequences06.json
符號序列預覽： oWWWoWWoWWWWWWWWWWWWWWooWWWWWoWWWWWWoWWWWWWWWoWWWWWWWoWWWWooooWWoWWWWWWWWWWoooWWWWoWWoWWWWWWooWoWWWooWoWWoWWoWWWWWWWoWWoWWWooWoWWWWoWWoWWWoWoWWWWWWWWWWoWWWWWWWWWWWWWWWWWWWWWoWWWWWWWoWWoWooWWWWWWWWWWWW
